# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: Unveiling the Power of DeepLearning.AI

Dive into the world of artificial intelligence with DeepLearning.AI. Offering top-notch courses created by industry experts, this platform equips you with the skills needed to thrive in the AI field. From neural networks to computer vision, DeepLearning.AI covers it all in an engaging and accessible way. Whether you're a beginner or looking to advance your knowledge, there's something for everyone. Join a global community of learners and take your first step towards mastering AI. Embrace the future with DeepLearning.AI - where innovation meets education. Start your journey today.


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Dive into the world of artificial intelligence with DeepLearning.AI. Offering top-notch courses created by industry experts, this platform equips you with the skills needed to thrive in the AI field. From neural networks to computer vision, DeepLearning.AI covers it all in an engaging and accessible way. Whether you're a beginner or looking to advance your knowledge, there's something for everyone. Join a global community of learners and take your first step towards mastering AI. Embrace the future with DeepLearning.AI - where innovation meets education. Start your journey today.

--------------------------------------------------------------------------------
Critic (to Writer):

Ove

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Dive into the world of artificial intelligence with DeepLearning.AI. Offering top-notch courses created by industry experts, this platform equips you with the skills needed to thrive in the AI field. From neural networks to computer vision, DeepLearning.AI covers it all in an engaging and accessible way. Whether you're a beginner or looking to advance your knowledge, there's something for everyone. Join a global community of learners and take your first step towards mastering AI. Embrace the future with DeepLearning.AI - where innovation meets education. Start your journey today.

--------------------------------------------------------------------------------

***********************

After aggregating feedback from all reviewers, here are the key points:

1. **SEO Reviewer**: The content should include relevant keywords, optimize meta tags, and add internal links for better SEO performance.
2. **Legal Reviewer**: Ensure compliance with advertising regulations, review the usage of trademarks, and confirm privacy and data protection compliance.
3. **Ethics Reviewer**: Verify that claims about DeepLearning.AI courses are backed by evidence, authenticate testimonials and success stories, and ensure compliance with data protection regulations.

Based on the feedback, here are some final suggestions for the writing:
- Incorporate relevant keywords, optimize meta tags, and add internal links to enhance SEO performance.
- Review and revise the content to comply with advertising regulations, trademark usage, and data protection laws.
- Provide verifiable evidence to support claims about DeepLearning.AI courses, authenticate testimonials, and ensure compliance with data prot

## Get the summary

In [15]:
print(res.summary)

Title: Mastering Artificial Intelligence with DeepLearning.AI

Unlock the realm of artificial intelligence through DeepLearning.AI's expert-led courses. From neural networks to computer vision, this platform caters to all proficiency levels. Join a global community, gain practical skills, and propel your AI career. Dive in today and embark on a journey towards AI mastery. Enhance SEO with strategic keywords, meta tags, and internal links. Ensure compliance with advertising and data protection laws. Substantiate course claims with evidence and validate testimonials ethically. Striking a balance between SEO, legalities, and ethics, DeepLearning.AI sets the stage for your AI success. Explore, learn, and excel.
